<a href="https://colab.research.google.com/github/liminovna/HSE_NLP_ASSIGNMENTS/blob/main/module2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способ заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import pandas as pd

In [2]:
! wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/notebooks/bow/labeled.csv.zip
!unzip -o labeled.csv.zip

--2024-11-23 11:19:52--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/notebooks/bow/labeled.csv.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/bow/labeled.csv.zip [following]
--2024-11-23 11:19:53--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/bow/labeled.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1463222 (1.4M) [application/zip]
Saving to: ‘labeled.csv.zip’

labeled.csv.zip     100%[===================>]   1.39M  --.-KB/s    in 0.05s   

2024-11-23 11:19:53 (28.7 M

In [3]:
data = pd.read_csv('labeled.csv')
data

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
# Замена токенайзера sklearn на кастомный.
# Переопределяем паттерн для токенизатора, чтобы за одно слово считались слова, написанные через дефис
vectorizer = CountVectorizer(token_pattern=r'(?u)\b[\w-]+\b')
X = vectorizer.fit_transform(data['comment'])
vectorizer.get_feature_names_out()

array(['0', '0-4', '00', ..., 'ёпта', 'ёта', 'ётавских'], dtype=object)

In [6]:
# обучаем векторайзер с дефолтной токенизацией
default_vectorizer = CountVectorizer()
default_X = default_vectorizer.fit_transform(data['comment'])
default_vectorizer.get_feature_names_out()

array(['00', '000', '0015', ..., 'ёпта', 'ёта', 'ётавских'], dtype=object)

In [ ]:
# Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def eval_logistic_w_counterv(data, target, tokenizer=None):
  # Делим данные на инпут и таргет
  X = data
  y = target

  # Делим датасет на обучающую и тестовую выборки.
  # На вход будут подаваться только лемматизированные заголовки+отзывы

  SEED = 42
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED)

  print('train and test split:', X_train.shape, X_test.shape)

  vectorizer = CountVectorizer(tokenizer=tokenizer)

  # Векторизируем обучающие данные
  matrix = vectorizer.fit_transform(X_train)

  # Векторизируем тренировочные данные
  matrix_test = vectorizer.transform(X_test)

  # Обучаем логистическую регрессию
  clf = LogisticRegression(random_state=SEED)
  clf.fit(matrix, y_train)
  pred = clf.predict(matrix_test) # Получаем предсказания для тестовой выборки

  # Метрики модели
  print(classification_report(y_test, pred))
  # return clf

In [8]:
eval_logistic_w_counterv(data['comment'],data['toxic'])

train and test split: (10809,) (3603,)
              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90      2444
         1.0       0.83      0.68      0.75      1159

    accuracy                           0.85      3603
   macro avg       0.85      0.81      0.82      3603
weighted avg       0.85      0.85      0.85      3603



In [9]:
# обучаем векторайзер с токенизацией razdel.tokenize
!pip install razdel

In [10]:
def custom_razdel_tokenizer(text):
  return [t.text for t in list(tokenize(text))]

In [11]:
from razdel import tokenize

razdel_vectorizer = CountVectorizer(tokenizer=custom_razdel_tokenizer)
razdel_X = razdel_vectorizer.fit_transform(data['comment'])
razdel_vectorizer.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array(['!', '!!', '!!!', ..., 'ёпта', 'ёта', 'ётавских'], dtype=object)

In [12]:
eval_logistic_w_counterv(data['comment'],data['toxic'], tokenizer=custom_razdel_tokenizer)

train and test split: (10809,) (3603,)
              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90      2444
         1.0       0.83      0.71      0.77      1159

    accuracy                           0.86      3603
   macro avg       0.85      0.82      0.83      3603
weighted avg       0.86      0.86      0.86      3603



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Можно сказать, что обе модели справились одинаково хорошо (плохо?): большой разницы в метриках нет

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [ ]:
# Логистическая регрессия с TfidfVectorizer

In [1]:
import pandas as pd
data = pd.read_csv('labeled.csv')
data

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Делим данные на инпут и таргет
X = data['comment']
y = data['toxic']

# Делим датасет на обучающую и тестовую выборки.
# На вход будут подаваться только лемматизированные заголовки+отзывы

SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED)

print('train and test split:', X_train.shape, X_test.shape)

train and test split: (10809,) (3603,)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.metrics import f1_score

In [16]:
vectorizer = TfidfVectorizer(
    stop_words=['и', 'что'],
    token_pattern=r'(?u)\b[\w]+\b',
    ngram_range=(1, 1),
    strip_accents='unicode',
    lowercase=True
)

# Векторизируем обучающие данные
matrix = vectorizer.fit_transform(X_train)

# Векторизируем тестовые данные
matrix_test = vectorizer.transform(X_test)

# Обучаем логистическую регрессию
clf = LogisticRegression(random_state=SEED, solver='lbfgs', penalty=None) # {‘lbfgs’, ‘liblinear’, ‘newton-cg’, ‘newton-cholesky’, ‘sag’, ‘saga’}
clf.fit(matrix, y_train)
pred = clf.predict(matrix_test) # Получаем предсказания для тестовой выборки

# Метрики модели
print(f1_score(y_test, pred))
# return clf

0.7732005432322318


In [15]:
pd.set_option('display.max_colwidth', None)

In [18]:
# Топ-10 токсичных комментариев для LogisticRegression
results = X_test.to_frame()
results['target'] = y_test
results['prediction'] = clf.predict_proba(matrix_test)[:,1]
results.sort_values('prediction', ascending=False).head(10)

,comment,target,prediction
2457,ну у хохлов еще хуже а мне-то что?\n,1.0,1.0
2582,"Бесплатный долбоеб, ты? Выйди нахуй из треда\n",1.0,1.0
14285,"Пиздец у быдла с пикабу сначала горело от негров на нулевой, теперь от скримеров, куда я нахуй попал, ебаные животные это БЭ, ЭТО РАНДОМ СУЧАРА, ТАМ НЕ ДОЛЖНО БЫТЬ ПРАВИЛ, ПОШЕЛ НАХУЙ\n",1.0,1.0
6497,"null 0 Сука, какие же коммибляди тупые.\n",1.0,1.0
5813,"Оп - хуесос и пидорас конченный, а его мать шлюха подзаборная.\n",1.0,1.0
2407,Забыл опиздюливание хохлов\n,1.0,1.0
381,"Типичный новиопский даун. А хохол без вышиванки для тебя не хохол, а узбек без тюбетейки не узбек?\n",1.0,1.0
5803,"бедная пидораха, и здесь хохлы ей выноваты\n",1.0,1.0
13763,"Ебать дебил, откуда вы абырвалги берётесь на дваче?\n",1.0,1.0
5623,Так воюют Русские А так воюют хохлы\n,1.0,1.0


In [ ]:
# Наивный Байес с CountVectorizer

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
vectorizer = CountVectorizer(
    stop_words=None,
    token_pattern=r'(?u)\b\w\w+\b',
    ngram_range=(1, 1),
    strip_accents=None,
    lowercase=True
)

gnb = MultinomialNB()

# Векторизируем обучающие данные
matrix = vectorizer.fit_transform(X_train)

# Векторизируем тестовые данные
matrix_test = vectorizer.transform(X_test)

# Обучаем MultinomialNB
gnb = MultinomialNB(fit_prior=False, alpha=1)
gnb.fit(matrix, y_train)
pred = gnb.predict(matrix_test) # Получаем предсказания для тестовой выборки

# Метрики модели
print(f1_score(y_test, pred))

0.7748091603053435


In [16]:
# Топ-10 токсичных комментариев для Naive Bayes
results = X_test.to_frame()
results['target'] = y_test
results['prediction'] = gnb.predict_proba(matrix_test)[:,1]
results.sort_values('prediction', ascending=False).head(10)

comment  \
6554                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

Топы 10-ти самых токсичных комментариев для обеих моделей полностью отличаются

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [24]:
pd.set_option('display.max_colwidth', 100)

In [17]:
import pandas as pd
data = pd.read_csv('labeled.csv')
data

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и сторонник демократии и свободы слова закукарекал.\n,1.0
14408,"А кого любить? Гоблина тупорылого что-ли? Или какую-нибудь продажную суку из демшизы? Нет уж, спасибо не надо.\n",1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, что это хороший фильм, такая высокобюджетная артхаусятина, к которой могут быть претензии только потому, что спиздили-распилили и вообще ТАК НЕ БЫВАЕТ. Ну нахуй этих критиков. Обзоры длиннее фильмов, петросянство хуже рашкокомедий, ебанутая ненависть и доебки по мелочам.\n",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ НЕТ ОБСУЖДЕНИЯ ПОЛИТИКИ. СВОБОДНОЕ ОБЩЕНИЕ ЭТО В b. ЭТО ТОЖЕ САМОЕ ЕСЛИ Я НА ДОСКЕ О ПОЛИТИКЕ СОЗДАМ ТРЕД О ТОМ КАК ПЛОХО БЕЗ ТЯНОЧКИ. ТАК ЧТО УЕБЫВАЙТЕ В Б ИЛИ НВР СО СВОИМ ЧАТИКОМ ПРЕСТАРЕЛЫХ ГОМОСЕКОВ!\n,1.0


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Делим данные на инпут и таргет
X = data['comment']
y = data['toxic']

# Делим датасет на обучающую и тестовую выборки.

SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED)

print('train and test split:', X_train.shape, X_test.shape)

train and test split: (10809,) (3603,)


In [19]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
stop_words.extend(
  ['это',
 'тебе',
 'ты',
 'вообще',
 'ещё',
 'всё',
 'почему',
 'которые',
 'какие'
 ]
)

In [ ]:
# Logistic Regression

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

vectorizer = CountVectorizer(
    # stop_words=stop_words,
    token_pattern=r'\b[\w]{3,}\b'
)

# Векторизируем обучающие данные
matrix = vectorizer.fit_transform(X_train)

# Векторизируем тренировочные данные
matrix_test = vectorizer.transform(X_test)

In [21]:
# Обучаем логистическую регрессию и задаем вручную параметры solver='lbfgs', penalty=None
clf = LogisticRegression(random_state=SEED, solver='lbfgs', penalty=None)
clf.fit(matrix, y_train)
pred = clf.predict(matrix_test) # Получаем предсказания для тестовой выборки

# Метрики модели
print(f1_score(y_test, pred))

0.7513321492007105


In [22]:
# топ-5 токсичных слов для логистической регрессии
weights = pd.DataFrame({'lemma': vectorizer.get_feature_names_out(),
                        'weight': clf.coef_.flatten()})
weights.sort_values('weight', ascending=False)[:5]

,lemma,weight
53337,хохлов,42.279232
53365,хохлы,36.439539
42572,русских,30.348953
47789,сука,29.658546
5053,быдло,29.108036


In [ ]:
# Decision Tree Classifier

In [61]:
from sklearn.tree import DecisionTreeClassifier

In [62]:
vectorizer = CountVectorizer(
    stop_words=stop_words,
    token_pattern=r'\b(?:[-\w]+|[!?]+)\b'
)

# Векторизируем обучающие данные
matrix = vectorizer.fit_transform(X_train)

# Векторизируем тренировочные данные
matrix_test = vectorizer.transform(X_test)

# Обучаем дерево решений и задаем вручную параметры min_samples_leaf=3, criterion='gini'
dtc = DecisionTreeClassifier(random_state=SEED, min_samples_leaf=3, criterion='gini') #{“gini”, “entropy”, “log_loss”}
dtc.fit(matrix, y_train)
pred = dtc.predict(matrix_test) # Получаем предсказания для тестовой выборки

# Метрики модели
print(f1_score(y_test, pred))

0.5877592311583207


In [63]:
# топ-5 токсичных слов для логистической регрессии
weights = pd.DataFrame({'lemma': vectorizer.get_feature_names_out(),
                        'weight': dtc.feature_importances_})
weights.sort_values('weight', ascending=False)[:5]

,lemma,weight
54555,хохлы,0.028892
54526,хохлов,0.021372
26631,нахуй,0.019889
31862,очень,0.015272
48880,сука,0.013040


In [ ]:
# RandomForestClassifier

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [64]:
vectorizer = CountVectorizer(
    stop_words=stop_words,
    # token_pattern=r'\b(?:[-\w]+|[!?]+)\b'
)

# Векторизируем обучающие данные
matrix = vectorizer.fit_transform(X_train)

# Векторизируем тренировочные данные
matrix_test = vectorizer.transform(X_test)

# Обучаем случайный лес и задаем вручную параметры min_samples_leaf=3, criterion='gini'
rfc = RandomForestClassifier(random_state=SEED, n_estimators=100, min_samples_split=10, max_features='log2')
rfc.fit(matrix, y_train)
pred = rfc.predict(matrix_test) # Получаем предсказания для тестовой выборки

# Метрики модели
print(f1_score(y_test, pred))

0.6507760532150776


In [65]:
weights = pd.DataFrame({'lemma': vectorizer.get_feature_names_out(),
                        'weight': rfc.feature_importances_})
weights.sort_values('weight', ascending=False)[:5]

,lemma,weight
53780,хохлы,0.005790
53752,хохлов,0.003842
31272,очень,0.002388
26067,нахуй,0.002100
4567,блядь,0.001931


In [ ]:
# MultinomialNB

In [59]:
vectorizer = CountVectorizer(
    stop_words=stop_words,
    token_pattern=r'(?u)\b\w\w+\b',
    ngram_range=(1, 1),
    strip_accents=None,
    lowercase=True
)

gnb = MultinomialNB()

# Векторизируем обучающие данные
matrix = vectorizer.fit_transform(X_train)

# Векторизируем тестовые данные
matrix_test = vectorizer.transform(X_test)

# Обучаем MultinomialNB и задаем вручную параметры fit_prior=True, alpha=1
gnb = MultinomialNB(fit_prior=True, alpha=1)
gnb.fit(matrix, y_train)
pred = gnb.predict(matrix_test) # Получаем предсказания для тестовой выборки

# Метрики модели
print(f1_score(y_test, pred))

0.7633885102239533


In [60]:
# топ-5 токсичных слов для MultinomialNB
weights = pd.DataFrame({'lemma': vectorizer.get_feature_names_out(),
                        'weight': gnb.feature_log_prob_[1] })
weights.sort_values('weight', ascending=False)[:5]

,lemma,weight
39450,просто,-5.929051
53780,хохлы,-6.945986
26067,нахуй,-6.996248
53752,хохлов,-7.188932
4568,блять,-7.214250


После нескольких попыток поменять параметры и векторайзеры для дерева решений и случайного леса метрику f1 не получилось поднять до 0.75